In [1]:
import torch.nn as nn
import numpy as np
from torch.utils.data.dataset import Dataset
import pandas as pd
import torch
import os
from utils import RunningMean
from sklearn.model_selection import StratifiedKFold
from torch.autograd import Variable
import utils
import datetime
import torchvision.transforms as transforms

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
date=str(datetime.date.today())
IMAGE_TRAIN_PRE='../data/AgriculturalDisease_trainingset/images/'
ANNOTATION_TRAIN='../data/AgriculturalDisease_trainingset/AgriculturalDisease_train_annotations_deleteNoise.json' #是否需要剔除两类异常类
IMAGE_VAL_PRE='../data/AgriculturalDisease_validationset/images/'
ANNOTATION_VAL='../data/AgriculturalDisease_validationset/AgriculturalDisease_validation_annotations_deleteNoise.json' #是否需要剔除两类异常类
with open(ANNOTATION_TRAIN) as datafile1:
    trainDataFram=pd.read_json(datafile1,orient='records')
with open(ANNOTATION_VAL) as datafile2: #first check if it's a valid json file or not
    validateDataFram =pd.read_json(datafile2,orient='records')

In [3]:
class StackCNN(nn.Module):
    def __init__(self,classNum=59):
        super(StackCNN,self).__init__()
        self.classNum=classNum
        self.features=nn.Sequential(
#             nn.Conv2d(1,16,kernel_size=(5,1)),
#             nn.ReLU(),
#             nn.Conv2d(16,32,kernel_size=(5,1)),
#             nn.ReLU(),
#             nn.Conv2d(32,64,kernel_size=(2,1)),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Conv2d(16,32,kernel_size=(4,1)),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Conv2d(32,64,kernel_size=(4,1)),
#             nn.ReLU(),
#             nn.Dropout(0.5)     
           
        )
        self.classifier=nn.Sequential(
            nn.Linear(9*59,2048),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(2048,1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512,self.classNum)
        )
    def forward(self,x):
     #   x=self.features(x)
        x=x.view(x.size(0),-1)
        x=self.classifier(x)
        return x
    


In [5]:
class ArrayLoader(Dataset):
    def __init__(self,img,label,transform=None):
        self.img=img
        self.label=label
        self.transform=transform
    def __len__(self):
        return len(self.label)
    def __getitem__(self, idx):
        idx_img=torch.from_numpy(np.transpose(self.img[idx]))
        idx_img=torch.unsqueeze(idx_img,0)
        if self.transform is not None:
            idx_img = self.transform(idx_img)
        label = torch.from_numpy(np.array(self.label[idx]))
        return idx_img, label
        

In [6]:
def load_data(featurePaths,dataFrame):
    label=dataFrame['disease_class'].values # array[1,1,1.....38]
    predictScoreBase=torch.unsqueeze(torch.load(featurePaths[0])['px'],2)
    print(predictScoreBase)
    predictScoreBase=predictScoreBase.numpy()
    for i in range(1,len(featurePaths)):
        predict=torch.load(featurePaths[i])
        predictScore=torch.unsqueeze(predict['px'],2).numpy()
        predictScoreBase=np.concatenate((predictScoreBase,predictScore),2)
        print('size of predict is ',predictScoreBase.shape)
    return predictScoreBase,label
    

In [7]:
def stacking(epochNum,featurenPaths,dataFrame):    
    train_val_x,train_val_y= load_data(featurenPaths,dataFrame)
    split_fold = StratifiedKFold(n_splits=4)
    folds_indexes=split_fold.split(train_val_x,train_val_y)
    folds_indexes= np.array(list(folds_indexes))
    i=folds_indexes[1]
    X_train=train_val_x[i[0],:,:]
    y_train=train_val_y[i[0]]
    X_test = train_val_x[i[1],:,:]
    y_test=train_val_y[i[1]]
    train_set=ArrayLoader(X_train,y_train)
    train_dataLoader=torch.utils.data.DataLoader(train_set,batch_size=32,shuffle=True,num_workers=4)
    val_set=ArrayLoader(X_test,y_test)
    val_dataLoader = torch.utils.data.DataLoader(val_set, batch_size = 32, shuffle=False, num_workers=4)
    model=StackCNN()
    model.cuda()
    criterion = nn.CrossEntropyLoss().cuda()    
    patience=0
    lr=1e-4
    min_loss=3.0
    min_acc=0.8
    for epoch in range(epochNum):
        
        if epoch==3:
            lr=1e-5
            momentum=0.9
            print('set lr=:%f,momentum=%f'%(lr,momentum))
        if  epoch<30 and epoch>3:
            lr=1e-4
        if epoch>=30 and epoch <100:
            lr=1e-4
        if epoch>=100:
            lr=1e-5
        
        if epoch==0 or epoch==1 or epoch==2: #第一轮首先训练全连接层
            lr=1e-5
#             optimizer=torch.optim.SGD(params=model.fresh_params(),lr=lr,momentum=0.9)
            optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
        else:
            
            optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))       
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'result', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'result', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)
        

In [8]:
featurePaths=['../feature/DesNet161/val_raw_prediction.pth','../feature/Xception/val_raw_prediction.pth','../feature/InceptionV4/val_raw_prediction.pth','../feature/Resnet152/val_raw_prediction.pth','../feature/Nasnetmobile/val_raw_prediction.pth','../feature/DesNet201/val_raw_prediction.pth','../feature/Resnet50/val_raw_prediction.pth','../feature/InceptionResnet/val_raw_prediction.pth','../feature/InceptionV3/val_raw_prediction.pth']
stacking(1000,featurePaths,validateDataFram)

tensor([[[ -6.3491],
         [  1.7410],
         [ -0.2039],
         ...,
         [-21.6580],
         [-20.3711],
         [-23.8674]],

        [[ -4.1716],
         [  2.4423],
         [ -0.6653],
         ...,
         [-23.9087],
         [-21.7002],
         [-22.8158]],

        [[ -9.7286],
         [ -0.4072],
         [  3.5345],
         ...,
         [-19.0716],
         [-14.1471],
         [-16.3393]],

        ...,

        [[-11.0720],
         [-11.1584],
         [-14.1832],
         ...,
         [-13.5836],
         [-15.4471],
         [-14.4771]],

        [[ -6.6540],
         [ -7.7115],
         [-12.0358],
         ...,
         [-14.6526],
         [-17.0814],
         [-14.7242]],

        [[-11.6538],
         [-13.4898],
         [-17.4383],
         ...,
         [-17.6303],
         [-19.4855],
         [-15.0748]]])
size of predict is  (4539, 59, 2)
size of predict is  (4539, 59, 3)
size of predict is  (4539, 59, 4)
size of predict is  (4539, 59, 5

[epoch:23,batch:89]:acc: 0.769792,loss:0.630256
[epoch:23] :acc: 0.767599,loss:0.625820,lr:0.000100,patience:0
[epoch:23]: val_loss:0.449211,val_acc:0.826923,
[epoch:24,batch:29]:acc: 0.790625,loss:0.611591
[epoch:24,batch:59]:acc: 0.786458,loss:0.593183
[epoch:24,batch:89]:acc: 0.782639,loss:0.594643
[epoch:24] :acc: 0.778498,loss:0.594854,lr:0.000100,patience:1
[epoch:24]: val_loss:0.431978,val_acc:0.834790,
save new model loss,now loss is  0.4319784641265869
save new model acc,now acc is  tensor(0.8348, device='cuda:0')
[epoch:25,batch:29]:acc: 0.773958,loss:0.593620
[epoch:25,batch:59]:acc: 0.774479,loss:0.602838
[epoch:25,batch:89]:acc: 0.773611,loss:0.605828
[epoch:25] :acc: 0.777614,loss:0.596435,lr:0.000100,patience:0
[epoch:25]: val_loss:0.435040,val_acc:0.828671,
[epoch:26,batch:29]:acc: 0.784375,loss:0.599846
[epoch:26,batch:59]:acc: 0.779687,loss:0.593993
[epoch:26,batch:89]:acc: 0.774306,loss:0.610323
[epoch:26] :acc: 0.776730,loss:0.612480,lr:0.000100,patience:1
[epoch:26

[epoch:53]: val_loss:0.434510,val_acc:0.837413,
[epoch:54,batch:29]:acc: 0.842708,loss:0.420611
[epoch:54,batch:59]:acc: 0.825000,loss:0.488533
[epoch:54,batch:89]:acc: 0.821181,loss:0.485452
[epoch:54] :acc: 0.819735,loss:0.483155,lr:0.000100,patience:14
[epoch:54]: val_loss:0.421470,val_acc:0.838287,
[epoch:55,batch:29]:acc: 0.821875,loss:0.472984
[epoch:55,batch:59]:acc: 0.813021,loss:0.486670
[epoch:55,batch:89]:acc: 0.825000,loss:0.455699
[epoch:55] :acc: 0.821208,loss:0.464254,lr:0.000100,patience:15
[epoch:55]: val_loss:0.414969,val_acc:0.836538,
[epoch:56,batch:29]:acc: 0.828125,loss:0.435161
[epoch:56,batch:59]:acc: 0.821354,loss:0.447835
[epoch:56,batch:89]:acc: 0.819444,loss:0.458531
[epoch:56] :acc: 0.814433,loss:0.478599,lr:0.000100,patience:16
[epoch:56]: val_loss:0.416216,val_acc:0.841783,
[epoch:57,batch:29]:acc: 0.822917,loss:0.484281
[epoch:57,batch:59]:acc: 0.810417,loss:0.487918
[epoch:57,batch:89]:acc: 0.818403,loss:0.463126
[epoch:57] :acc: 0.824153,loss:0.451092,

[epoch:85,batch:89]:acc: 0.832986,loss:0.432081
[epoch:85] :acc: 0.835935,loss:0.429906,lr:0.000100,patience:45
[epoch:85]: val_loss:0.415496,val_acc:0.847028,
[epoch:86,batch:29]:acc: 0.827083,loss:0.450349
[epoch:86,batch:59]:acc: 0.833854,loss:0.443380
[epoch:86,batch:89]:acc: 0.835764,loss:0.435281
[epoch:86] :acc: 0.838586,loss:0.424665,lr:0.000100,patience:46
[epoch:86]: val_loss:0.413921,val_acc:0.843531,
[epoch:87,batch:29]:acc: 0.826042,loss:0.450380
[epoch:87,batch:59]:acc: 0.834375,loss:0.433600
[epoch:87,batch:89]:acc: 0.840972,loss:0.415267
[epoch:87] :acc: 0.845361,loss:0.417070,lr:0.000100,patience:47
[epoch:87]: val_loss:0.412662,val_acc:0.847028,
[epoch:88,batch:29]:acc: 0.817708,loss:0.433931
[epoch:88,batch:59]:acc: 0.832292,loss:0.415753
[epoch:88,batch:89]:acc: 0.831597,loss:0.426410
[epoch:88] :acc: 0.837113,loss:0.418620,lr:0.000100,patience:48
[epoch:88]: val_loss:0.417053,val_acc:0.836538,
[epoch:89,batch:29]:acc: 0.833333,loss:0.440204
[epoch:89,batch:59]:acc:

[epoch:116]: val_loss:0.417823,val_acc:0.850524,
[epoch:117,batch:29]:acc: 0.881250,loss:0.316520
[epoch:117,batch:59]:acc: 0.870833,loss:0.334392
[epoch:117,batch:89]:acc: 0.870486,loss:0.329517
[epoch:117] :acc: 0.871870,loss:0.320751,lr:0.000010,patience:77
[epoch:117]: val_loss:0.421665,val_acc:0.849650,
[epoch:118,batch:29]:acc: 0.847917,loss:0.378176
[epoch:118,batch:59]:acc: 0.860938,loss:0.341482
[epoch:118,batch:89]:acc: 0.863889,loss:0.328751
[epoch:118] :acc: 0.863328,loss:0.334401,lr:0.000010,patience:78
[epoch:118]: val_loss:0.418666,val_acc:0.851399,
[epoch:119,batch:29]:acc: 0.870833,loss:0.302962
[epoch:119,batch:59]:acc: 0.872396,loss:0.305916
[epoch:119,batch:89]:acc: 0.869097,loss:0.326169
[epoch:119] :acc: 0.874227,loss:0.317046,lr:0.000010,patience:79
[epoch:119]: val_loss:0.422693,val_acc:0.847028,
[epoch:120,batch:29]:acc: 0.882292,loss:0.288501
[epoch:120,batch:59]:acc: 0.872396,loss:0.333550
[epoch:120,batch:89]:acc: 0.873264,loss:0.341686
[epoch:120] :acc: 0.8

...

[epoch:872,batch:89]:acc: 0.923611,loss:0.208476
[epoch:872] :acc: 0.922533,loss:0.212208,lr:0.000010,patience:832
[epoch:872]: val_loss:0.545045,val_acc:0.849650,
[epoch:873,batch:29]:acc: 0.918750,loss:0.204215
[epoch:873,batch:59]:acc: 0.918229,loss:0.208099
[epoch:873,batch:89]:acc: 0.922569,loss:0.206317
[epoch:873] :acc: 0.922533,loss:0.214759,lr:0.000010,patience:833
[epoch:873]: val_loss:0.558262,val_acc:0.846154,
[epoch:874,batch:29]:acc: 0.912500,loss:0.219851
[epoch:874,batch:59]:acc: 0.918750,loss:0.213594
[epoch:874,batch:89]:acc: 0.914931,loss:0.220085
[epoch:874] :acc: 0.916642,loss:0.214645,lr:0.000010,patience:834
[epoch:874]: val_loss:0.546570,val_acc:0.853147,
[epoch:875,batch:29]:acc: 0.916667,loss:0.223164
[epoch:875,batch:59]:acc: 0.920833,loss:0.214571
[epoch:875,batch:89]:acc: 0.921181,loss:0.209724
[epoch:875] :acc: 0.919293,loss:0.214055,lr:0.000010,patience:835
[epoch:875]: val_loss:0.546019,val_acc:0.847028,
[epoch:876,batch:29]:acc: 0.919792,loss:0.188786
[

[epoch:903]: val_loss:0.562929,val_acc:0.854895,
[epoch:904,batch:29]:acc: 0.914583,loss:0.222336
[epoch:904,batch:59]:acc: 0.915104,loss:0.214663
[epoch:904,batch:89]:acc: 0.917361,loss:0.210452
[epoch:904] :acc: 0.916937,loss:0.212470,lr:0.000010,patience:864
[epoch:904]: val_loss:0.560794,val_acc:0.848776,
[epoch:905,batch:29]:acc: 0.916667,loss:0.209155
[epoch:905,batch:59]:acc: 0.920312,loss:0.203233
[epoch:905,batch:89]:acc: 0.919792,loss:0.199681
[epoch:905] :acc: 0.920177,loss:0.194989,lr:0.000010,patience:865
[epoch:905]: val_loss:0.566972,val_acc:0.853147,
[epoch:906,batch:29]:acc: 0.925000,loss:0.197603
[epoch:906,batch:59]:acc: 0.920833,loss:0.201236
[epoch:906,batch:89]:acc: 0.923958,loss:0.199306
[epoch:906] :acc: 0.924006,loss:0.196912,lr:0.000010,patience:866
[epoch:906]: val_loss:0.553180,val_acc:0.846154,
[epoch:907,batch:29]:acc: 0.918750,loss:0.209682
[epoch:907,batch:59]:acc: 0.916146,loss:0.215138
[epoch:907,batch:89]:acc: 0.919097,loss:0.211382
[epoch:907] :acc: 

[epoch:935,batch:89]:acc: 0.924306,loss:0.188907
[epoch:935] :acc: 0.925479,loss:0.186147,lr:0.000010,patience:895
[epoch:935]: val_loss:0.588891,val_acc:0.851399,
[epoch:936,batch:29]:acc: 0.927083,loss:0.172192
[epoch:936,batch:59]:acc: 0.926562,loss:0.184082
[epoch:936,batch:89]:acc: 0.921528,loss:0.193702
[epoch:936] :acc: 0.919882,loss:0.196470,lr:0.000010,patience:896
[epoch:936]: val_loss:0.562536,val_acc:0.847028,
[epoch:937,batch:29]:acc: 0.927083,loss:0.192774
[epoch:937,batch:59]:acc: 0.932813,loss:0.189868
[epoch:937,batch:89]:acc: 0.929167,loss:0.196150
[epoch:937] :acc: 0.929308,loss:0.194432,lr:0.000010,patience:897
[epoch:937]: val_loss:0.563964,val_acc:0.844406,
[epoch:938,batch:29]:acc: 0.920833,loss:0.181967
[epoch:938,batch:59]:acc: 0.912500,loss:0.202108
[epoch:938,batch:89]:acc: 0.913889,loss:0.211603
[epoch:938] :acc: 0.914875,loss:0.209236,lr:0.000010,patience:898
[epoch:938]: val_loss:0.552697,val_acc:0.847028,
[epoch:939,batch:29]:acc: 0.909375,loss:0.211267
[

[epoch:966]: val_loss:0.567083,val_acc:0.843531,
[epoch:967,batch:29]:acc: 0.921875,loss:0.197522
[epoch:967,batch:59]:acc: 0.921354,loss:0.195965
[epoch:967,batch:89]:acc: 0.924306,loss:0.195201
[epoch:967] :acc: 0.926068,loss:0.193269,lr:0.000010,patience:927
[epoch:967]: val_loss:0.574451,val_acc:0.850524,
[epoch:968,batch:29]:acc: 0.922917,loss:0.184768
[epoch:968,batch:59]:acc: 0.921354,loss:0.202281
[epoch:968,batch:89]:acc: 0.918056,loss:0.203728
[epoch:968] :acc: 0.917820,loss:0.201803,lr:0.000010,patience:928
[epoch:968]: val_loss:0.570258,val_acc:0.847028,
[epoch:969,batch:29]:acc: 0.925000,loss:0.198191
[epoch:969,batch:59]:acc: 0.923958,loss:0.194693
[epoch:969,batch:89]:acc: 0.922917,loss:0.195060
[epoch:969] :acc: 0.921060,loss:0.200488,lr:0.000010,patience:929
[epoch:969]: val_loss:0.567258,val_acc:0.842657,
[epoch:970,batch:29]:acc: 0.919792,loss:0.215348
[epoch:970,batch:59]:acc: 0.913542,loss:0.223309
[epoch:970,batch:89]:acc: 0.915278,loss:0.209978
[epoch:970] :acc: 

[epoch:998,batch:89]:acc: 0.921875,loss:0.185503
[epoch:998] :acc: 0.922239,loss:0.188583,lr:0.000010,patience:958
[epoch:998]: val_loss:0.569140,val_acc:0.849650,
[epoch:999,batch:29]:acc: 0.933333,loss:0.193438
[epoch:999,batch:59]:acc: 0.927604,loss:0.199314
[epoch:999,batch:89]:acc: 0.928472,loss:0.197450
[epoch:999] :acc: 0.928719,loss:0.198337,lr:0.000010,patience:959
[epoch:999]: val_loss:0.568821,val_acc:0.855769,
